# __IMPORT PACKAGES__
---

In [ ]:
# -*- coding=utf-8 -*-
# __author = 'bonneting'__

""" 
@author: bonnieting
@contact: dingbangchu@gmail.com
"""

# basic:
import datetime as dt
import os, sys
import pandas as pd
pd.set_option('display.max_columns', 10000, 'display.max_rows', 10000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import numpy as np
np.set_printoptions(suppress=True, threshold=np.nan)
import warnings
warnings.filterwarnings('ignore')
import re
from openpyxl.styles import numbers
from string import ascii_uppercase
from common import db_util, report_util
import copy

# gragh:
from plotly.offline import init_notebook_mode
import plotly.offline as py
init_notebook_mode(connected=True)
import plotly.graph_objects as go
import plotly.figure_factory as ff
from IPython.display import Markdown, display, HTML
import seaborn as sns
import plotly as ply
ply.offline.init_notebook_mode(connected=True)

# time:
from pytz import timezone
import datetime
from dateutil.relativedelta import relativedelta
import calendar
from dateutil.parser import parse

In [ ]:
pre-define
today = dt.datetime.today()
current_month = today.strftime('%Y-%m')
start_date = (today.date() - relativedelta(days=7)).strftime('%Y-%m-%d')

day = calendar.monthrange(today.year,today.month)[1]
end_date = dt.datetime(today.year ,today.month,day).strftime('%Y-%m-%d') 

print('today:'+str(today),
      'current_month:'+str(current_month),
      'start_date:'+str(start_date),
      'end_date:'+str(end_date))
print('本月天数：'+str(day))

In [ ]:
# define a function to get the first day and last day in a month (need check every running)
def this_month_last_day(year=None, month=None):
    if year:        
        year = int(year)
    else:        
        year = dt.date.today().year
    if month:        
        month = int(month)
    else:        
        month = dt.date.today().month  

    x, y = calendar.monthrange(year, month) # x：days of week of the first day of that month # y：total days of that month

    first_day = dt.date(year=year, month=month, day=1)  
    last_day = dt.date(year=year, month=month, day=y)
    
    return first_day, last_day

first_day, last_day = this_month_last_day()
first_day=str(first_day)
last_day=str(last_day)

print(first_day,last_day)
# first_day, last_day = func(2023,6)
# print(first_day, last_day)
  
# year = dt.date.today().year
t_month = dt.date.today()+relativedelta(months=-1)
t_year=t_month.year
t_month = t_month.month
x, y = calendar.monthrange(t_year, t_month)
this_month=str(dt.date(year=t_year, month=t_month, day=y))

l_month=dt.date.today()+relativedelta(months=-2)
l_month=l_month.month
x, y = calendar.monthrange(t_year, l_month)
last_month=str(dt.date(year=t_year, month=l_month, day=y))

this_week=str(dt.date.today()+relativedelta(weeks=-1,weekday=calendar.MONDAY)) # weeks should +=1 if it's Monday when u run this code
last_week=str(dt.date.today()+relativedelta(weeks=-2,weekday=calendar.MONDAY)) # same as above

# dont need to modify each week
print(this_month,last_month,this_week,last_week)

# __PRODUCE DATAFRAMES__
---
- _with no specific format_

## OVERVIEW
### transaction

In [ ]:
def calc(df, index, sort_column=None):
    df = df.drop(['type'],axis=1)
    df = df.pivot_table(index = index,columns = 'period', fill_value=0).stack(0).reset_index()
    if sort_column == None:
        df = df.sort_index(axis=1,ascending=False)
    else:
        df = df.sort_values(sort_column,ascending=False).sort_index(axis=1,ascending=False)
    
    return df

In [ ]:
@db_util.sql_to_df('ofp_mysql')
def data_overview_new1():
    """transaction"""
    sql = """    
    select 
        'week' as type
        , date_format(subdate(date(order_end_time),if(date_format(date(order_end_time),'%%w')=0,7,date_format(date(order_end_time),'%%w'))-1),'%%Y-%%m-%%d') as period
        , cast(worth_currency as char) as worth_currency
        , count(DISTINCT order_id) as total_case_cnt
        , count(DISTINCT cid) as total_customer_cnt_cumu
        , sum(transaction_volume) as total_amount
        , sum(transaction_volume)/count(DISTINCT order_id) as avg_case_amt
        , count(DISTINCT order_id)/count(DISTINCT cid) as avg_customer_case
        , sum(transaction_volume)/count(DISTINCT cid) as avg_customer_amt
    from retail.ba_succ_transaction  
    where order_end_time>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day 
    and order_end_time<current_date and worth_currency='CNY'
    group by 1,2,3
    union all
    select 
        'month' as type
        , date_format(last_day(order_end_time),'%%Y-%%m-%%d') as period
        , cast(worth_currency as char) as worth_currency
        , count(DISTINCT order_id) as total_case_cnt
        , count(DISTINCT cid) as total_customer_cnt_cumu
        , sum(transaction_volume) as total_amount
        , sum(transaction_volume)/count(DISTINCT order_id) as avg_case_amt
        , count(DISTINCT order_id)/count(DISTINCT cid) as avg_customer_case
        , sum(transaction_volume)/count(DISTINCT cid) as avg_customer_amt
    from retail.ba_succ_transaction  
    where order_end_time>=date_format(current_date,'%%Y-%%m-01') - interval '7' month 
    and order_end_time<current_date
    and worth_currency='CNY'
    group by 1,2,3
    order by 1,2 desc,3
    """
    return sql

In [ ]:
## transaction
df_transaction = data_overview_new1() # rawdata
df_transaction_week = df_transaction[(df_transaction['type']=='week')].append(df_transaction.head(1)) #  & (df_transaction['period']==end_date)
df_transaction_week = calc(df_transaction_week,index = 'worth_currency',sort_column = 'level_1')
df_transaction_week=df_transaction_week.rename(columns={'level_1':'including OTC'},inplace=False)
df_transaction_week["vsLastWeek"]=df_transaction_week.apply(lambda x: '%.2f%%' % ((x[this_week]-x[last_week])*100/x[last_week]), axis=1)
df_transaction_week.insert(3,'vs.LastWeek',df_transaction_week.pop('vsLastWeek'),allow_duplicates=False)
df_transaction_week.drop(columns='worth_currency',inplace=True)
df_transaction_week.set_index('including OTC',inplace=True)


df_transaction_month = df_transaction[(df_transaction['type']=='month') & (df_transaction['period']!=end_date)]
df_transaction_month = calc(df_transaction_month,index = 'worth_currency',sort_column = 'level_1')
df_transaction_month=df_transaction_month.rename(columns={'level_1':'including OTC'},inplace=False)
df_transaction_month["vsLastMonth"]=df_transaction_month.apply(lambda x: '%.2f%%' % ((x[this_month]-x[last_month])*100/x[last_month]), axis=1)
df_transaction_month.insert(2,'vs.LastMonth',df_transaction_month.pop('vsLastMonth'),allow_duplicates=False)
df_transaction_month.drop(columns='worth_currency',inplace=True)
df_transaction_month.set_index('including OTC',inplace=True)
# df_transaction
# df_transaction_week
# df_transaction_month

# list(map(lambda x,y: function(x,y), df['col1'], df['col2']))
# df_transaction_week['vsLastWeek']=list(map(lambda x,y: x/y,df_transaction_week[df_transaction_week.iloc[:,[3]]],df_transaction_week[df_transaction_week.iloc[:,[4]]]))
# df_transaction_week.insert(3,'vs.LastWeek',value=a)
# df_transaction_week["vsLastWeek"]=(df_transaction_week.iloc[:,[3]]/df_transaction_week.iloc[:,[4]])###.values
# col = df_transaction_week.columns.tolist()
# col.insert(col[3],'vs.LastWeek')
# df.reindex(columns=col)

In [ ]:
df_transaction_week

In [ ]:
df_transaction_month

### balance 

In [ ]:
@db_util.sql_to_df('ofp_mysql')
def data_overview_new2():
    """balance"""
    sql = """    
    select 
        'week' as type
        , date_format(subdate(date(order_end_time),if(date_format(date(order_end_time),'%%w')=0,7,date_format(date(order_end_time),'%%w'))-1),'%%Y-%%m-%%d') as period
        , is_otc
        , worth_currency
        , record_type
        , sum(transaction_volume) as tx_amt_CNY
        , sum(transaction_volume / price) as tx_amt_CNY
    from retail.ba_succ_transaction m1 
    join retail.dim_exchange_rate m2 on m2.settle_date=date(m1.order_end_time) and currency_code='CNY'
    where order_end_time>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day and order_end_time<current_date
    and order_type in ('Fiat_DW','COIN_IN_OUT') and worth_currency=m2.legal_code
    and m1.worth_currency='CNY'
    group by 1,2,3,4,5
    union all
    select 
        'month' as type
        , date_format(last_day(order_end_time),'%%Y-%%m-%%d') as period
        , is_otc
        , worth_currency
        , record_type
        , sum(transaction_volume) as tx_amt_CNY
        , sum(transaction_volume / price) as tx_amt_CNY
    from retail.ba_succ_transaction m1
    join retail.dim_exchange_rate m2 on m2.settle_date=date(m1.order_end_time) and currency_code='CNY'
    where order_end_time>=date_format(current_date,'%%Y-%%m-01') - interval '7' month and order_end_time<current_date
    and order_type in ('Fiat_DW','COIN_IN_OUT') and worth_currency=m2.legal_code
    and m1.worth_currency='CNY'
    group by 1,2,3,4,5
    order by 1,4,3,5,2 desc 
    """
    return sql

## balance
df_balance = data_overview_new2() 
df_balance = df_balance.replace({'DEPOSIT':'1_DEPOSIT','COIN_IN':'2_COIN_IN',
                             'WITHDRAW':'3_WITHDRAW','COIN_OUT':'4_COIN_OUT'}) # rawdata

In [ ]:
df_balance_week = df_balance[(df_balance['type'] == 'week')|(df_balance['period'] == end_date)] # .append(df_balance[])
df_balance_week = calc(df_balance_week,index = ['worth_currency','is_otc','record_type'],sort_column = 'level_3')
df_balance_week = df_balance_week.rename(columns={'level_3':'*NON-OTC'})

df_balance_week["vsLastWeek"]=df_balance_week.apply(lambda x: '%.2f%%' % ((x[this_week]-x[last_week])*100/x[last_week]), axis=1)
# df_balance_week["vsLastWeek"]=(df_balance_week.iloc[:,[5]].values/df_transaction_week.iloc[:,[6]]).values
df_balance_week.insert(4,'vs.LastWeek',df_balance_week.pop('vsLastWeek'),allow_duplicates=False)
df_balance_week.drop(columns=['worth_currency','is_otc'],inplace=True)
df_balance_week.set_index(['record_type','*NON-OTC'],inplace=True)

df_balance_month = df_balance[(df_balance['type'] == 'month')] # & (df_balance['period'] != end_date)]
df_balance_month = calc(df_balance_month,index = ['worth_currency','is_otc','record_type'],sort_column = 'level_3')
df_balance_month=df_balance_month.rename(columns={'level_3':'*NON-OTC'}) 
# format(x, '.2%') --> str
df_balance_month["vsLastMonth"]=df_balance_month.apply(lambda x: '%.2f%%' % ((x[this_month]-x[last_month])*100/x[last_month]), axis=1)
df_balance_month.insert(4,'vs.LastMonth',df_balance_month.pop('vsLastMonth'),allow_duplicates=False)
df_balance_month.drop(columns=['worth_currency','is_otc'],inplace=True)
df_balance_month.set_index(['record_type','*NON-OTC'],inplace=True)
# df_balance_month["vsLastMonth"]=df_balance_month.apply(lambda x: (x.iloc[:,4:4]-x.iloc[:,5:5])/x.iloc[:,5:5],axis=1)
# (df_balance_week.iloc[:,6]-df_balance_week.iloc[:,7])/df_balance_week.iloc[:,7]


# df_balance_week.loc['Net_CashIn_CNY']=''
# df_balance_week.loc['Net_CashIn']=''
df_balance_week.iloc[8,1:]=df_balance_week.iloc[0,1:]+df_balance_week.iloc[1,1:]-df_balance_week.iloc[2,1:]-df_balance_week.iloc[3,1:]
# (df_balance_week.loc[1].values[0:]+df_balance_week.loc[3].values[0:]-df_balance_week.loc[5].values[0:]-df_balance_week.loc[7].values[0:])
df_balance_week.iloc[9,1:]=df_balance_week.iloc[4,1:]+df_balance_week.iloc[5,1:]-df_balance_week.iloc[6,1:]-df_balance_week.iloc[7,1:]
# (df_balance_week.loc[0].values[0:]+df_balance_week.loc[2].values[0:]-df_balance_week.loc[4].values[0:]-df_balance_week.loc[6].values[0:])
df_balance_week.loc['Net_CashIn_CNY'].replace('','-',inplace=True)
df_balance_week.loc['Net_CashIn'].replace('','-',inplace=True)

df_balance_month.loc['Net_CashIn_CNY']=''
df_balance_month.loc['Net_CashIn']=''
df_balance_month.iloc[8,1:]=df_balance_month.iloc[0,1:]+df_balance_month.iloc[1,1:]-df_balance_month.iloc[2,1:]-df_balance_month.iloc[3,1:]
# (df_balance_month.loc[1].values[0:]+df_balance_month.loc[3].values[0:]-df_balance_month.loc[5].values[0:]-df_balance_month.loc[7].values[0:])
df_balance_month.iloc[9,1:]=df_balance_month.iloc[4,1:]+df_balance_month.iloc[5,1:]-df_balance_month.iloc[6,1:]-df_balance_month.iloc[7,1:]
# (df_balance_month.loc[0].values[0:]+df_balance_month.loc[2].values[0:]-df_balance_month.loc[4].values[0:]-df_balance_month.loc[6].values[0:])
df_balance_month.loc['Net_CashIn_CNY'].replace('','-',inplace=True)
df_balance_month.loc['Net_CashIn'].replace('','-',inplace=True)

In [ ]:
df_balance_week

In [ ]:
df_balance_month

## REGISTER & CONVERSION
### conversion detail

In [ ]:
@db_util.sql_to_df('ofp_mysql')
def data_overview_new3():
    """check the customers from Google"""
    sql = """    
    select 
        'week' as type
        , date_format(subdate(date(m1.register_time),if(date_format(date(m1.register_time),'%%w')=0,7,date_format(date(m1.register_time),'%%w'))-1),'%%Y-%%m-%%d') as period
        , market_source
        , register_device_type
        , m1.rna_area
        , m1.rna_type
        , count(DISTINCT m1.cid) as register_counts
        , count(DISTINCT if(subdate(date(min_rna1_submit_time),if(date_format(date(min_rna1_submit_time),'%%w')=0,7,date_format(date(min_rna1_submit_time),'%%w'))-1) = subdate(date(register_time),if(date_format(date(register_time),'%%w')=0,7,date_format(date(register_time),'%%w'))-1),m1.cid,null)) as period_try_rna1_counts
        , count(DISTINCT if(subdate(date(min_rna1_succ_time),if(date_format(date(min_rna1_succ_time),'%%w')=0,7,date_format(date(min_rna1_succ_time),'%%w'))-1) = subdate(date(register_time),if(date_format(date(register_time),'%%w')=0,7,date_format(date(register_time),'%%w'))-1),m1.cid,null)) as period_rna1_counts 
        , count(DISTINCT if(subdate(date(min_try_deposit_time),if(date_format(date(min_try_deposit_time),'%%w')=0,7,date_format(date(min_try_deposit_time),'%%w'))-1) = subdate(date(register_time),if(date_format(date(register_time),'%%w')=0,7,date_format(date(register_time),'%%w'))-1),m1.cid,null)) as period_try_deposit_counts
        , count(DISTINCT if(subdate(date(min_trade_all_time),if(date_format(date(min_trade_all_time),'%%w')=0,7,date_format(date(min_trade_all_time),'%%w'))-1) = subdate(date(register_time),if(date_format(date(register_time),'%%w')=0,7,date_format(date(register_time),'%%w'))-1),m1.cid,null)) as period_tot_counts
        , count(DISTINCT if(subdate(date(min_deposit_time),if(date_format(date(min_deposit_time),'%%w')=0,7,date_format(date(min_deposit_time),'%%w'))-1) = subdate(date(register_time),if(date_format(date(register_time),'%%w')=0,7,date_format(date(register_time),'%%w'))-1),m1.cid,null)) as period_deposit_counts
        , count(DISTINCT if(subdate(date(min_trade_time),if(date_format(date(min_trade_time),'%%w')=0,7,date_format(date(min_trade_time),'%%w'))-1) = subdate(date(register_time),if(date_format(date(register_time),'%%w')=0,7,date_format(date(register_time),'%%w'))-1),m1.cid,null)) as period_trade_counts 
    from retail.ba_customer m1 
    where m1.register_time>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day 
    and m1.register_time<current_date
    and m1.rna_area !='AMERICA'
    group by 1,2,3,4,5,6
    union all
    select 
        'month' as type
        , date_format(last_day(m1.register_time),'%%Y-%%m-%%d') as period
        , market_source
        , register_device_type
        , m1.rna_area
        , m1.rna_type
        , count(DISTINCT m1.cid) as register_counts
        , count(DISTINCT if(date_format(m1.min_rna1_submit_time,'%%Y-%%m')=date_format(m1.register_time,'%%Y-%%m'),m1.cid,null)) as period_try_rna1_counts
        , count(DISTINCT if(date_format(m1.min_rna1_succ_time,'%%Y-%%m')=date_format(m1.register_time,'%%Y-%%m'),m1.cid,null)) as period_rna1_counts
        , count(DISTINCT if(date_format(m1.min_try_deposit_time,'%%Y-%%m')=date_format(m1.register_time,'%%Y-%%m'),m1.cid,null)) as period_try_deposit_counts
        , count(DISTINCT if(date_format(m1.min_trade_all_time,'%%Y-%%m')=date_format(m1.register_time,'%%Y-%%m'),m1.cid,null)) as period_tot_counts
        , count(DISTINCT if(date_format(m1.min_deposit_time,'%%Y-%%m')=date_format(m1.register_time,'%%Y-%%m'),m1.cid,null)) as period_deposit_counts
        , count(DISTINCT if(date_format(m1.min_trade_time,'%%Y-%%m')=date_format(m1.register_time,'%%Y-%%m'),m1.cid,null)) as period_trade_counts
    from retail.ba_customer m1 
    where m1.register_time>=date_format(current_date,'%%Y-%%m-01') - interval '7' month 
    and m1.register_time<current_date
    and m1.rna_area !='AMERICA'
    group by 1,2,3,4,5,6
    order by 1,2 desc,3
    """
    return sql

## customer acquisition & conv_funnel
df_conv_detail = data_overview_new3()
# df_conv_detail

In [ ]:
df_conv_detail[(df_conv_detail['period'] == '2022-01-10') & (df_conv_detail['market_source'] == 'Google_app')]

In [ ]:
df_conv_detail_ov_week=df_conv_detail[df_conv_detail['type']=='week'].pivot_table(index=['period'],
                                             values=['register_counts','period_rna1_counts','period_deposit_counts','period_trade_counts','period_tot_counts'],
                                             aggfunc=[np.sum],
                                             fill_value=0,
                                             margins=True,
                                             margins_name='TOTAL',
                                             sort=True
                                            ).sort_index(axis=0,ascending=False).sort_index(axis=1,ascending=False)
df_conv_detail_ov_month=df_conv_detail[df_conv_detail['type']=='month'].pivot_table(index=['period'],
                                             values=['register_counts','period_rna1_counts','period_deposit_counts','period_trade_counts','period_tot_counts'],
                                             aggfunc=[np.sum],
                                             fill_value=0,
                                             margins=True,
                                             margins_name='TOTAL',
                                             sort=True
                                            ).sort_index(axis=0,ascending=False).sort_index(axis=1,ascending=False)

df_conv_detail_ov_week
df_conv_detail_ov_month

In [ ]:
# compute on rows & cols

## modify the weekly data
# df_conv_detail_ov_week.columns.names=['func','cols']
df_conv_detail_ov_week=df_conv_detail_ov_week.xs('sum',axis=1)
df_conv_detail_ov_week['a.reg2rna1_succ']=df_conv_detail_ov_week['period_rna1_counts']/df_conv_detail_ov_week['register_counts']
df_conv_detail_ov_week['b.reg2depo_succ']=df_conv_detail_ov_week['period_deposit_counts']/df_conv_detail_ov_week['register_counts']
df_conv_detail_ov_week['c.reg2trade_succ']=df_conv_detail_ov_week['period_trade_counts']/df_conv_detail_ov_week['register_counts']
df_conv_detail_ov_week.drop(axis=1,columns=['period_rna1_counts','period_deposit_counts','period_trade_counts'],inplace=True)
df_conv_detail_ov_week.loc["vsLastWeek"]=df_conv_detail_ov_week.apply(lambda x: '%.2f%%' % ((x.loc[this_week]-x.loc[last_week])*100/x.loc[last_week]),axis=0)
df_conv_detail_ov_week.iloc[0:8,2:5]=df_conv_detail_ov_week.iloc[0:8,2:5].applymap(lambda x: '%.2f%%' % (x*100))
# ！！！make it to 9 if it's not Monday !!!
# [['a.reg2rna1_succ','b.reg2depo_succ','c.reg2trade_succ']].iloc[0:9]
df_conv_detail_ov_week=df_conv_detail_ov_week.T
df_conv_detail_ov_week.insert(1,'vs.LastWeek',df_conv_detail_ov_week.pop('vsLastWeek'),allow_duplicates=False)

# df['stock_money'] = df['fund_size'].astype(
#     float) * df['stock_percent'].str.strip('%').astype(float) / 100
# ).format(x, '.2%')

In [ ]:
df_conv_detail_ov_week

In [ ]:
## modify monthly data
df_conv_detail_ov_month=df_conv_detail_ov_month.xs('sum',axis=1)
df_conv_detail_ov_month['a.reg2rna1_succ']=df_conv_detail_ov_month['period_rna1_counts']/df_conv_detail_ov_month['register_counts']
df_conv_detail_ov_month['b.reg2depo_succ']=df_conv_detail_ov_month['period_deposit_counts']/df_conv_detail_ov_month['register_counts']
df_conv_detail_ov_month['c.reg2trade_succ']=df_conv_detail_ov_month['period_trade_counts']/df_conv_detail_ov_month['register_counts']
df_conv_detail_ov_month.drop(axis=1,columns=['period_rna1_counts','period_deposit_counts','period_trade_counts'],inplace=True)
df_conv_detail_ov_month.loc["vsLastMonth"]=df_conv_detail_ov_month.apply(lambda x: '%.2f%%' % ((x.loc[this_month]-x.loc[last_month])*100/x.loc[last_month]),axis=0)
df_conv_detail_ov_month.iloc[0:8,2:5]=df_conv_detail_ov_month.iloc[0:8,2:5].applymap(lambda x: '%.2f%%' % (x*100))
# [['a.reg2rna1_succ','b.reg2depo_succ','c.reg2trade_succ']].iloc[0:9]
df_conv_detail_ov_month=df_conv_detail_ov_month.T
df_conv_detail_ov_month.insert(1,'vs.LastMonth',df_conv_detail_ov_month.pop('vsLastMonth'),allow_duplicates=False)

In [ ]:
df_conv_detail_ov_month

### first trade overview 

In [ ]:
@db_util.sql_to_df('ofp_mysql')
def data_overview_new4():
    """distinguish by trade_currency and market_source"""
    sql = """    
    select 
        'week' as type
        , cast(subdate(date(m1.order_end_time),if(date_format(date(m1.order_end_time),'%%w')=0,7,date_format(date(m1.order_end_time),'%%w'))-1) as char) as period
        , worth_currency
        , market_source
        , count(DISTINCT m1.cid) as new_user_cnt
    from retail.ba_dwp_first_trade_all m1 
    left join retail.ba_customer m2 on m2.cid=m1.cid 
    where m1.order_end_time>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day 
    and m1.order_end_time<current_date
    and m2.rna_area !='AMERICA'
    group by 1,2,3,4
    union all
    select 
        'month' as type
        , cast(date_format(last_day(m1.order_end_time),'%%Y-%%m-%%d') as char) as period
        , worth_currency 
        , market_source
        , count(DISTINCT m1.cid) as new_user_cnt
    from retail.ba_dwp_first_trade_all m1 
    left join retail.ba_customer m2 on m2.cid=m1.cid 
    where m1.order_end_time>=date_format(current_date,'%%Y-%%m-01') - interval '7' month 
    and m1.order_end_time<current_date
    and m2.rna_area !='AMERICA'
    group by 1,2,3,4
    order by 1,2,3 desc,4
    """
    return sql

df_first_trade = data_overview_new4()

df_first_trade_month = df_first_trade[(df_first_trade['type']=='month')|(df_first_trade['period'] == end_date)]
df_first_trade_month = calc(df_first_trade_month,index = ['worth_currency','market_source']).drop('level_2',axis=1)

df_first_trade_week = df_first_trade[(df_first_trade['type']=='week')&(df_first_trade['period'] != end_date)]
df_first_trade_week = calc(df_first_trade_week,index = ['worth_currency','market_source']).drop('level_2',axis=1)

df_first_trade

In [ ]:
df_first_trade_month

In [ ]:
df_first_trade_week

In [ ]:
df_first_trade_month_ttl=df_first_trade_month.pivot_table(
                                             index=['worth_currency'],
#                                              values=all,
                                             aggfunc=[np.sum],
                                             fill_value=0,
                                             margins=True,
                                             margins_name='period_first_trade_all',
                                             sort=True
                                            ).sort_index(axis=1,ascending=False).xs('sum',axis=1)
# .sort_index(axis=0,ascending=False)
# df_conv_detail_ov_week=df_conv_detail_ov_week.xs('sum',axis=1)
df_first_trade_month_ttl.drop(axis=0,index='CNY',inplace=True)
df_first_trade_month_ttl['TOTAL'] = df_first_trade_month_ttl.apply(func=np.sum,axis=1)
df_first_trade_month_ttl.insert(0,'TOTAL',df_first_trade_month_ttl.pop('TOTAL'),allow_duplicates=False)

df_first_trade_week_ttl=df_first_trade_week.pivot_table(
                                             index=['worth_currency'],
                                             aggfunc=[np.sum],
                                             fill_value=0,
                                             margins=True,
                                             margins_name='period_first_trade_all',
                                             sort=True
                                            ).sort_index(axis=1,ascending=False).xs('sum',axis=1)
df_first_trade_week_ttl.drop(axis=0,index='CNY',inplace=True)
df_first_trade_week_ttl['TOTAL'] = df_first_trade_week_ttl.apply(func=np.sum,axis=1)
df_first_trade_week_ttl.insert(0,'TOTAL',df_first_trade_week_ttl.pop('TOTAL'),allow_duplicates=False)

In [ ]:
df_first_trade_month_ttl

In [ ]:
df_first_trade_week_ttl

In [ ]:
df_first_trade_month_ttl["vsLastMonth"]=df_first_trade_month_ttl.apply(lambda x: '%.2f%%' % ((x[this_month]-x[last_month])*100/x[last_month]),axis=1)
df_first_trade_month_ttl.insert(0,'vs.LastMonth',df_first_trade_month_ttl.pop('vsLastMonth'),allow_duplicates=False)

df_first_trade_week_ttl["vsLastWeek"]=df_first_trade_week_ttl.apply(lambda x: '%.2f%%' % ((x[this_week]-x[last_week])*100/x[last_week]),axis=1)
df_first_trade_week_ttl.insert(0,'vs.LastWeek',df_first_trade_week_ttl.pop('vsLastWeek'),allow_duplicates=False)
# df_conv_detail_ov_week.loc['first_trade_counts'].iloc[:,1:] # =df_first_trade_month_ttl.iloc[0]

df_conv_detail_ov_week=df_conv_detail_ov_week.append(df_first_trade_week_ttl,ignore_index=False)
df_conv_detail_ov_month=df_conv_detail_ov_month.append(df_first_trade_month_ttl,ignore_index=False)#！！still need test！！change to false for no deleting the former names

In [ ]:
df_conv_detail_ov_month

In [ ]:
df_conv_detail_ov_week

### channel conv detail

In [ ]:
df_channel_week=df_conv_detail[df_conv_detail['type']=='week'].pivot_table(index=['period','market_source'],
                                             values=['register_counts','period_rna1_counts','period_deposit_counts','period_trade_counts','period_tot_counts','period_try_rna1_counts','period_try_deposit_counts'],
                                             aggfunc=[np.sum],
                                             fill_value=0,
                                             margins=True,
                                             margins_name='TOTAL',
                                             sort=True
                                            ).sort_index(axis=0,ascending=False).sort_index(axis=1,ascending=False)
df_channel_week=df_channel_week.xs('sum',axis=1)
df_channel_week=df_channel_week[df_channel_week['register_counts']>5]
df_channel_week['a.reg2rna1_succ']=df_channel_week['period_rna1_counts']/df_channel_week['register_counts']
df_channel_week['b.reg2depo_succ']=df_channel_week['period_deposit_counts']/df_channel_week['register_counts']
df_channel_week['c.reg2trade_succ']=df_channel_week['period_trade_counts']/df_channel_week['register_counts']
df_channel_week['d.reg2try_rna1(try/reg)']=df_channel_week['period_try_rna1_counts']/df_channel_week['register_counts']
df_channel_week['e.rna_succ(succ/try)']=df_channel_week['period_rna1_counts']/df_channel_week['period_try_rna1_counts']
df_channel_week['f.reg2try_depo(try/reg)']=df_channel_week['period_try_deposit_counts']/df_channel_week['register_counts']
df_channel_week['g.depo_succ(succ/try)']=df_channel_week['period_deposit_counts']/df_channel_week['period_try_deposit_counts']
df_channel_week.drop(axis=1,columns=['period_rna1_counts','period_deposit_counts','period_trade_counts','period_try_rna1_counts','period_try_deposit_counts'],inplace=True)
df_channel_week

In [ ]:
df_channel_month=df_conv_detail[df_conv_detail['type']=='month'].pivot_table(index=['period','market_source'],
                                             values=['register_counts','period_rna1_counts','period_deposit_counts','period_trade_counts','period_tot_counts','period_try_rna1_counts','period_try_deposit_counts'],
                                             aggfunc=[np.sum],
                                             fill_value=0,
                                             margins=True,
                                             margins_name='TOTAL',
                                             sort=True
                                            ).sort_index(axis=0,ascending=False).sort_index(axis=1,ascending=False)
df_channel_month=df_channel_month.xs('sum',axis=1)
df_channel_month=df_channel_month[df_channel_month['register_counts']>5]
df_channel_month['a.reg2rna1_succ']=df_channel_month['period_rna1_counts']/df_channel_month['register_counts']
df_channel_month['b.reg2depo_succ']=df_channel_month['period_deposit_counts']/df_channel_month['register_counts']
df_channel_month['c.reg2trade_succ']=df_channel_month['period_trade_counts']/df_channel_month['register_counts']
df_channel_month['d.reg2try_rna1(try/reg)']=df_channel_month['period_try_rna1_counts']/df_channel_month['register_counts']
df_channel_month['e.rna_succ(succ/try)']=df_channel_month['period_rna1_counts']/df_channel_month['period_try_rna1_counts']
df_channel_month['f.reg2try_depo(try/reg)']=df_channel_month['period_try_deposit_counts']/df_channel_month['register_counts']
df_channel_month['g.depo_succ(succ/try)']=df_channel_month['period_deposit_counts']/df_channel_month['period_try_deposit_counts']
df_channel_month.drop(axis=1,columns=['period_rna1_counts','period_deposit_counts','period_trade_counts','period_try_rna1_counts','period_try_deposit_counts'],inplace=True)
df_channel_month

## TRANSACTION
### overview cnt&amt

In [ ]:
# @db_util.sql_to_df('ofp_mysql')
def data_trans_type_new3():
    """distinguish by trade_device"""
    sql = """    
    select 
        'week' as type
        , date_format(subdate(date(order_end_time),if(date_format(date(order_end_time),'%%w')=0,7,date_format(date(order_end_time),'%%w'))-1),'%%Y-%%m-%%d') as period
        , worth_currency
        , order_device_type
        , count(DISTINCT if(record_type='DEPOSIT',order_id,null)) as depo_cnt
        , count(DISTINCT if(record_type='WITHDRAW',order_id,null)) as withd_cnt
        , count(DISTINCT if(record_type='COIN_IN',order_id,null)) as depo_times
        , count(DISTINCT if(record_type='COIN_OUT',order_id,null)) as withd_times
        , count(DISTINCT if(record_type in ('SELL','BUY'),order_id,null)) as tra_cnt
        , count(DISTINCT if(order_type='Agent_Fiat_TX',order_id,null)) as agent_cnt
        , count(DISTINCT if(order_type='Orderbook_Fiat_TX',order_id,null)) as distribution_cnt
        , count(DISTINCT if(order_type='Crypto_TX',order_id,null)) as coupon
    from retail.ba_succ_transaction
    where is_otc != 'OTC' and worth_currency='CNY'
    and order_end_time>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day and order_end_time<current_date 
    group by 1,2,3,4
    union all
    select 
        'month' as type
        , date_format(order_end_time,'%%Y-%%m') as period
        , worth_currency
        , order_device_type
        , count(DISTINCT if(record_type='DEPOSIT',order_id,null)) as depo_cnt
        , count(DISTINCT if(record_type='WITHDRAW',order_id,null)) as withd_cnt
        , count(DISTINCT if(record_type='COIN_IN',order_id,null)) as depo_times
        , count(DISTINCT if(record_type='COIN_OUT',order_id,null)) as withd_times
        , count(DISTINCT if(record_type in ('SELL','BUY'),order_id,null)) as tra_cnt
        , count(DISTINCT if(order_type='Agent_Fiat_TX',order_id,null)) as agent_cnt
        , count(DISTINCT if(order_type='Orderbook_Fiat_TX',order_id,null)) as distribution_cnt
        , count(DISTINCT if(order_type='Crypto_TX',order_id,null)) as coupon
    from retail.ba_succ_transaction
    where is_otc != 'OTC' and worth_currency='CNY'
    and order_end_time>=date_format(current_date,'%%Y-%%m-01') - interval '7' month 
    and order_end_time<current_date 
    group by 1,2,3,4
    order by 1,2
    """
    return sql

# df_order_device = data_trans_type_new3()

@db_util.sql_to_df('ofp_mysql')
def data_trans_type_new4():
    """distinguish by trade or not"""
    sql = """    
    select 
        'week' as type
        , date_format(subdate(date(order_end_time),if(date_format(date(order_end_time),'%%w')=0,7,date_format(date(order_end_time),'%%w'))-1),'%%Y-%%m-%%d') as period
        , is_otc
        , worth_currency
        , count(DISTINCT cid) as tx_customer_cnt
        , sum(transaction_volume) as tx_amount
    from retail.ba_succ_transaction
    where order_end_time>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day 
    and order_end_time<current_date
    and worth_currency='CNY'
    group by 1,2,3,4
    union all
    select 
        'month' as type
        , date_format(last_day(order_end_time),'%%Y-%%m-%%d') as period
        , is_otc
        , worth_currency
        , count(DISTINCT cid) as tx_customer_cnt
        , sum(transaction_volume) as tx_amount
    from retail.ba_succ_transaction
    where order_end_time>=date_format(curdate(),'%%Y-%%m-01') - interval '7' month 
    and order_end_time<current_date
    and worth_currency='CNY'
    group by 1,2,3,4
    order by 1,2 desc
    """
    return sql

df_order_otc = data_trans_type_new4()

df_order_otc

df_order_otc_month = df_order_otc[(df_order_otc['type'] == 'month')] #  & (df_order_otc['period'] != current_month)

df_order_otc_month = df_order_otc_month.pivot_table(index=['worth_currency','is_otc'],
                                                    columns='period',
                                                    values=['tx_customer_cnt','tx_amount'],
                                                    aggfunc='sum').stack(0).sort_index(axis=1,ascending=False) # stack(0) change col.index to inner row.index
df_order_otc_month = df_order_otc_month.reset_index()
# drop(('CNY','OTC', 'tx_customer_cnt'),axis=0).
df_order_otc_month = df_order_otc_month.sort_values(['level_2','worth_currency'],ascending=[False,False])

df_order_otc_week = df_order_otc[(df_order_otc['type'] == 'week') | (df_order_otc['period'] == end_date)]
df_order_otc_week = df_order_otc_week.pivot_table(index=['worth_currency','is_otc'],columns='period',
                                                  values=['tx_customer_cnt','tx_amount'],
                                                  aggfunc='sum').stack(0).sort_index(axis=1,ascending=False)

df_order_otc_week = df_order_otc_week.reset_index().sort_values(['level_2','worth_currency'],ascending=[False,False])

# df_order_otc

In [ ]:
df_order_otc_month

In [ ]:
df_order_otc_week

### trans type

In [ ]:
@db_util.sql_to_df('ofp_mysql')
def data_trans_type_new1():
    """distinguish by trade_type"""
    sql = """    
    select type,period,worth_currency,ifnull(trans_type,'total')trans_type,tx_case_cnt,tx_customer_cnt,tx_amount
    from 
    (
        select 
            'week' as type
            , date_format(subdate(date(order_end_time),if(date_format(date(order_end_time),'%%w')=0,7,date_format(date(order_end_time),'%%w'))-1),'%%Y-%%m-%%d') as period
            , worth_currency
            , case 
                when is_otc ='OTC' then 'OTC'
                when is_otc != 'OTC' and sub_order_type='api' then concat(order_type,'_',sub_order_type)
                when is_otc != 'OTC' and (sub_order_type !='api' or sub_order_type is null) then order_type 
            end as trans_type
            , count(DISTINCT order_id) as tx_case_cnt
            , count(DISTINCT cid) as tx_customer_cnt
            , sum(transaction_volume) as tx_amount
        from retail.ba_succ_transaction
        where order_end_time>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day 
        and order_end_time<current_date and record_type in ('SELL','BUY')
        and worth_currency='CNY'
        group by 1,2,3,4
        union all
        select 
            'week' as type
            , date_format(subdate(date(order_end_time),if(date_format(date(order_end_time),'%%w')=0,7,date_format(date(order_end_time),'%%w'))-1),'%%Y-%%m-%%d') as period
            , worth_currency
            , 'total' as trans_type
            , count(DISTINCT order_id) as tx_case_cnt
            , count(DISTINCT cid) as tx_customer_cnt
            , sum(transaction_volume) as tx_amount
        from retail.ba_succ_transaction
        where order_end_time>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day 
        and order_end_time<current_date and record_type in ('SELL','BUY')
        and worth_currency='CNY'
        group by 1,2,3,4
        union all
        select 
            'month' as type
            , date_format(last_day(order_end_time),'%%Y-%%m-%%d') as period
            , worth_currency
            , case 
                when is_otc ='OTC' then 'OTC'
                when is_otc != 'OTC' and sub_order_type='api' then concat(order_type,'_',sub_order_type)
                when is_otc != 'OTC' and (sub_order_type !='api' or sub_order_type is null) then order_type 
            end as trans_type
            , count(DISTINCT order_id) as tx_case_cnt
            , count(DISTINCT cid) as tx_customer_cnt
            , sum(transaction_volume) as tx_amount
        from retail.ba_succ_transaction
        where order_end_time>=date_format(curdate(),'%%Y-%%m-01') - interval '7' month 
        and order_end_time<current_date and record_type in ('SELL','BUY')
        and worth_currency='CNY'
        group by 1,2,3,4
        union all
        select 
            'month' as type
            , date_format(last_day(order_end_time),'%%Y-%%m-%%d') as period
            , worth_currency
            , 'total'as trans_type
            , count(DISTINCT order_id) as tx_case_cnt
            , count(DISTINCT cid) as tx_customer_cnt
            , sum(transaction_volume) as tx_amount
        from retail.ba_succ_transaction
        where order_end_time>=date_format(curdate(),'%%Y-%%m-01') - interval '7' month 
        and order_end_time<current_date and record_type in ('SELL','BUY')
        and worth_currency='CNY'
        group by 1,2,3,4
    )a 
    where period is not null and worth_currency is not null
    order by 1,3,2 desc ,4
    """
    return sql

# 按交易类型
df_trans_type = data_trans_type_new1()

# df_trans_type

In [ ]:
# rearrange rows & cols of rawdata and compute vs.LastWeek/Month
df_trans_type_week=df_trans_type[(df_trans_type['type']=='week')|(df_trans_type['period']==end_date)].pivot_table(
                                             index=['trans_type','period'],
                                             values=['tx_customer_cnt','tx_amount'],
                                             fill_value=0,
                                             sort=True
                                            ).unstack(1).stack(0).swaplevel().sort_index(axis=0,ascending=False).sort_index(axis=1,ascending=False)

tx_amt_pct_week=df_trans_type_week.apply(lambda x: (x.loc['tx_amount']*100)/x.loc['tx_amount','total'])
tx_amt_pct_week.index=pd.MultiIndex.from_product([['tx_amt_pct'],
                        ['total', 'OrderBook_Fiat_TX_api', 'OrderBook_Fiat_TX', 'Crypto_TX_api','Crypto_TX', 'Agent_Fiat_TX']])
df_trans_type_week=df_trans_type_week.append(tx_amt_pct_week,verify_integrity=True,ignore_index=False)

avg_tx_amt_peruser_week=df_trans_type_week.apply(lambda x: (x.loc['tx_amount']/x.loc['tx_customer_cnt']))
avg_tx_amt_peruser_week.index=pd.MultiIndex.from_product([['avg_tx_amt_peruser'],
                        ['total', 'OrderBook_Fiat_TX_api', 'OrderBook_Fiat_TX', 'Crypto_TX_api','Crypto_TX', 'Agent_Fiat_TX']])
df_trans_type_week=df_trans_type_week.append(avg_tx_amt_peruser_week,verify_integrity=True,ignore_index=False)

df_trans_type_week['vsLastWeek']=df_trans_type_week.apply(lambda x: '%.2f%%' % ((x[this_week]-x[last_week])*100/x[last_week]),axis=1)
df_trans_type_week.iloc[12:18,:-1]=tx_amt_pct_week.applymap(lambda x: '%.2f%%' % x)
# df_trans_type_week.iloc[18:,:-1]=avg_tx_amt_peruser_week.applymap(lambda x: '%.2f%%' % x)
# ！！multiple index & slice rows and cols：.loc['trade_amt_pct'].iloc[:,:-1]
df_trans_type_week.insert(0,'vs.LastWeek',df_trans_type_week.pop('vsLastWeek'),allow_duplicates=False)




df_trans_type_month=df_trans_type[(df_trans_type['type']=='month')].pivot_table(
                                             index=['trans_type','period'],
                                             values=['tx_customer_cnt','tx_amount'],
                                             fill_value=0,
                                             sort=True
                                            ).unstack(1).stack(0).swaplevel().sort_index(axis=0,ascending=False).sort_index(axis=1,ascending=False)
tx_amt_pct_month=df_trans_type_month.apply(lambda x: (x.loc['tx_amount']*100)/x.loc['tx_amount','total'])
# trade_amt_pct_month=trade_amt_pct_month.applymap(lambda x: '%.2f%%' % x)
tx_amt_pct_month.index=pd.MultiIndex.from_product([['tx_amt_pct'],
                        ['total', 'OrderBook_Fiat_TX_api', 'OrderBook_Fiat_TX', 'Crypto_TX_api',
       'Crypto_TX', 'Agent_Fiat_TX']])
df_trans_type_month=df_trans_type_month.append(tx_amt_pct_month,verify_integrity=True,ignore_index=False)

avg_tx_amt_peruser_month=df_trans_type_month.apply(lambda x: (x.loc['tx_amount']/x.loc['tx_customer_cnt']))
avg_tx_amt_peruser_month.index=pd.MultiIndex.from_product([['avg_tx_amt_peruser'],
                        ['total', 'OrderBook_Fiat_TX_api', 'OrderBook_Fiat_TX', 'Crypto_TX_api',
       'Crypto_TX', 'Agent_Fiat_TX']])
df_trans_type_month=df_trans_type_month.append(avg_tx_amt_peruser_month,verify_integrity=True,ignore_index=False)

# only can do it when it's int/float: compite vslastweek and str() and then add "%" to that part, move vslastweek columns in the end
df_trans_type_month['vsLastMonth']=df_trans_type_month.apply(lambda x: '%.2f%%' % ((x[this_month]-x[last_month])*100/x[last_month]),axis=1)
df_trans_type_month.iloc[12:18,:-1]=tx_amt_pct_month.applymap(lambda x: '%.2f%%' % x)
# df_trans_type_month.iloc[18:,:-1]=avg_tx_amt_peruser_month.applymap(lambda x: '%.2f%%' % x)
df_trans_type_month.insert(0,'vs.LastMonth',df_trans_type_month.pop('vsLastMonth'),allow_duplicates=False)

In [ ]:
df_trans_type_week

In [ ]:
df_trans_type_month

### trans symbol

In [ ]:
@db_util.sql_to_df('ofp_mysql')
def data_trans_type_new2():
    """distinguish by trade_currency"""
    sql = """    
    select 
        'week' as type
        , date_format(subdate(date(order_end_time),if(date_format(date(order_end_time),'%%w')=0,7,date_format(date(order_end_time),'%%w'))-1),'%%Y-%%m-%%d') as period
        -- , order_type
        , if(symbol like '%%CNY%%','CNY','NON_CNY') as symbol
        , symbol as subsymbol
        , count(DISTINCT order_id) as case_cnt
        , sum(transaction_volume) as amount
    from retail.ba_succ_transaction
    where record_type in ('SELL','BUY') and is_otc != 'OTC' 
    and order_end_time>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day 
    and order_end_time<current_date 
    and worth_currency='CNY'
    group by 1,2,3,4
    union all
    select
        'month' as type
        , date_format(last_day(order_end_time),'%%Y-%%m-%%d') as period
        -- , order_type
        , if(symbol like '%%CNY%%','CNY','NON_CNY') as symbol
        , symbol as subsymbol
        , count(DISTINCT order_id) as case_cnt
        , sum(transaction_volume) as amount
    from retail.ba_succ_transaction
    where record_type in ('SELL','BUY') and is_otc != 'OTC' 
    and order_end_time>=date_format(curdate(),'%%Y-%%m-01') - interval '7' month 
    and order_end_time<date_format(curdate(),'%%Y-%%m-01')
    and worth_currency='CNY'
    group by 1,2,3,4
    order by 1,2 desc,6 desc
    """
    return sql


## 按买卖币对，待写出：每个月只取交易额前五的币对 SQL
df_symbol = data_trans_type_new2()

# df_symbol

In [ ]:
df_symbol_temp=pd.DataFrame(df_symbol.groupby(['period']).sum())
# df_symbol_temp2=pd.DataFrame(df_symbol.groupby(['period']).head(3).set_index('period'))
df_symbol_temp2=pd.DataFrame(df_symbol.groupby(['period']).apply(lambda x: x.sort_values('amount', ascending=False).head(3))).drop('period',axis=1)
df_symbol_temp2=df_symbol_temp2.join(df_symbol_temp,on='period',how='left',rsuffix='_ttl')
# df_symbol_temp2
# df_symbol_week_top=pd.DataFrame(df_symbol_temp2[df_symbol_temp2['type']=='week'].groupby(['period','subsymbol']))
df_symbol_week_top=df_symbol_temp2[df_symbol_temp2['type']=='week'].pivot_table(
#     values=['amount','amt_pct','case_cnt','case_pct'],
    index=['period','subsymbol'],
    aggfunc=np.sum,
    sort=True).sort_index(axis=0,ascending=False)
df_symbol_week_top['amt_pct']=df_symbol_week_top.apply(lambda x: '%.2f%%' % ((x['amount']*100)/x['amount_ttl']),axis=1)
df_symbol_week_top['case_pct']=df_symbol_week_top.apply(lambda x: '%.2f%%' % ((x['case_cnt']*100)/x['case_cnt_ttl']),axis=1)
df_symbol_week_top.drop(columns=['amount_ttl','case_cnt_ttl'],inplace=True)
# df_symbol_week_top.sort_values(by=['amount'],ascending=True)

# df_symbol['subsymbol'].groupby(df_symbol['period'],axis=0).['subsymbol'].rank(method='first',ascending=True))
df_symbol_month_top=df_symbol_temp2[df_symbol_temp2['type']=='month'].pivot_table(
    index=['period','subsymbol'],
    aggfunc=np.sum,
    sort=True).sort_index(axis=0,ascending=False)
df_symbol_month_top['amt_pct']=df_symbol_month_top.apply(lambda x: '%.2f%%' % ((x['amount']*100)/x['amount_ttl']),axis=1)
df_symbol_month_top['case_pct']=df_symbol_month_top.apply(lambda x: '%.2f%%' % ((x['case_cnt']*100)/x['case_cnt_ttl']),axis=1)
df_symbol_month_top.drop(columns=['amount_ttl','case_cnt_ttl'],inplace=True)
# df_symbol_month_top.sort_values(by=['amount'],ascending=True)

In [ ]:
df_symbol_week_top

In [ ]:
df_symbol_month_top

## REAL_NAME AUTHENTICATION

In [ ]:

@db_util.sql_to_df('ofp_mysql')
def data_rna2():
    """rna in different devices"""
    sql = """    
    select 
        'week' as type,
        date_format(subdate(date(m1.created_at),if(date_format(date(m1.created_at),'%%w')=0,7,date_format(date(m1.created_at),'%%w'))-1),'%%Y-%%m-%%d') as period,
        if(m1.source_mark='undefined','web',m1.source_mark) as source_mark,
        count(DISTINCT m1.id) as total,
        count(DISTINCT if(status = 'SUCCESSFUL',m1.id,null)) as suc
    from retail.ofp_rna_audit_record m1
    left join retail.ofp_customer_ext m2 on m2.cid = m1.cid
    where m1.created_at>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day and m1.created_at<current_date
    and m2.source <> 'TEST' and m1.rna_level=1 and rna_area='CHINA' and rna_type='INDIVIDUAL'
    group by 1,2,3
    union all
    select 
        'month' as type,
        date_format(m1.created_at,'%%Y-%%m') as period,
        if(m1.source_mark='undefined','web',m1.source_mark) as source_mark,
        count(DISTINCT m1.id) as total,
        count(DISTINCT if(status = 'SUCCESSFUL',m1.id,null)) as suc
    from retail.ofp_rna_audit_record m1
    left join retail.ofp_customer_ext m2 on m2.cid = m1.cid
    where m1.created_at>=date_format(current_date,'%%Y-%%m-01') - interval '7' month and m1.created_at<current_date
    and m2.source <> 'TEST' and m1.rna_level=1 and rna_area='CHINA' and rna_type='INDIVIDUAL'
    group by 1,2,3
    order by 3,2,1

    """
    return sql

@db_util.sql_to_df('ofp_mysql')
def data_rna3():
    """rna in different markets"""
    sql = """    
    select 
        'week' as type,
        date_format(subdate(date(m1.created_at),if(date_format(date(m1.created_at),'%%w')=0,7,date_format(date(m1.created_at),'%%w'))-1),'%%Y-%%m-%%d') as period,
        market_source,
        count(DISTINCT m1.id) as total,
        count(DISTINCT if(status = 'SUCCESSFUL',m1.id,null)) as suc
    from retail.ofp_rna_audit_record m1
    left join retail.ba_customer m2 on m2.cid = m1.cid
    where m1.created_at>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day and m1.created_at<current_date
    and m1.rna_level=1 and m1.rna_area='CHINA' and m1.rna_type='INDIVIDUAL'
    group by 1,2,3
    union all
    select 
        'month' as type,
        date_format(m1.created_at,'%%Y-%%m') as period,
        market_source,
        count(DISTINCT m1.id) as total,
        count(DISTINCT if(status = 'SUCCESSFUL',m1.id,null)) as suc
    from retail.ofp_rna_audit_record m1
    left join retail.ba_customer m2 on m2.cid = m1.cid
    where m1.created_at>=date_format(current_date,'%%Y-%%m-01') - interval '7' month and m1.created_at<current_date
    and m1.rna_level=1 and m1.rna_area='CHINA' and m1.rna_type='INDIVIDUAL'
    group by 1,2,3
    order by 3,2,1

    """
    return sql

In [ ]:
df_rna_device = data_rna2()

In [ ]:
df_rna_market = data_rna3()

In [ ]:
@db_util.sql_to_df('ofp_mysql')
def data_rna1():
    """rna succ_rate and timecost"""
    sql = """    
    select 
        'week' as type,
        date_format(subdate(date(m1.created_at),if(date_format(date(m1.created_at),'%%w')=0,7,date_format(date(m1.created_at),'%%w'))-1),'%%Y-%%m-%%d') as period,
        m1.rna_level,
        m1.rna_area,
        m1.rna_type,
        -- m1.auditor,
        case when m1.auditor='Ewally' then 'Ewally'
             when m1.auditor='IDWall' then 'IDWall'
             else 'normal'
        end as auditor_tag,
        count(DISTINCT m1.id) as total,
        count(DISTINCT m1.cid) as user,
        count(DISTINCT if(status = 'SUCCESSFUL',m1.id,null)) as suc,
        count(DISTINCT if(status = 'SUCCESSFUL',m1.cid,null)) as suc_cnt,
        count(DISTINCT m3.cid) as trade_all,
        count(DISTINCT m3.trade_cid) as trade,
        count(DISTINCT if(status = 'FAILED',m1.id,null)) as fail,
        count(DISTINCT if(status = 'FAILED',m1.cid,null)) as fail_cnt,
        count(DISTINCT if(status in ('PENDING', 'AUDITING'),m1.id,null)) as pending,
        sum(if(status in ('SUCCESSFUL','FAILED'),TIMESTAMPDIFF(minute,m1.created_at,m1.audited_at),0)) as total_deal_min
    from retail.ofp_rna_audit_record m1
    left join retail.ofp_customer_ext m2 on m2.cid = m1.cid
    left join (select cid,if(record_type in ('SELL','BUY'),cid,null) as trade_cid from retail.ba_succ_transaction where is_otc !='OTC' group by 1)m3 on m3.cid=m1.cid
    where m1.created_at>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '42' day and m1.created_at<current_date
    and m2.source <> 'TEST' and m1.rna_area !='AMERICA'
    and m1.status !='CANCELED'
    group by 1,2,3,4,5,6
    union all
    select 
        'month' as type,
        date_format(m1.created_at,'%%Y-%%m') as period,
        m1.rna_level,
        m1.rna_area,
        m1.rna_type,
        -- m1.auditor,
        case when m1.auditor='Ewally' then 'Ewally'
             when m1.auditor='IDWall' then 'IDWall'dwd_
             else 'normal'
        end as auditor_tag,
        count(DISTINCT m1.id) as total,
        count(DISTINCT m1.cid) as user,
        count(DISTINCT if(status = 'SUCCESSFUL',m1.id,null)) as suc,
        count(DISTINCT if(status = 'SUCCESSFUL',m1.cid,null)) as suc_cnt,
        count(DISTINCT m3.cid) as trade_all,
        count(DISTINCT m3.trade_cid) as trade,
        count(DISTINCT if(status = 'FAILED',m1.id,null)) as fail,
        count(DISTINCT if(status = 'FAILED',m1.cid,null)) as fail_cnt,
        count(DISTINCT if(status in ('PENDING', 'AUDITING'),m1.id,null)) as pending,
        sum(if(status in ('SUCCESSFUL','FAILED'),TIMESTAMPDIFF(minute,m1.created_at,m1.audited_at),0)) as total_deal_min
    from retail.ofp_rna_audit_record m1
    left join retail.ofp_customer_ext m2 on m2.cid = m1.cid
    left join (select cid,if(record_type in ('SELL','BUY'),cid,null) as trade_cid from retail.ba_succ_transaction where is_otc !='OTC' group by 1)m3 on m3.cid=m1.cid
    where m2.source <> 'TEST' and m1.rna_area !='AMERICA'
    and m1.status !='CANCELED'
    and m1.created_at>=date_format(curdate(),'%%Y-%%m-01') - interval '7' month 
    and m1.created_at<date_format(curdate(),'%%Y-%%m-01') 
    group by 1,2,3,4,5,6
    order by 1,3,5,6,2 desc
    """
    return sql

# rna audit_succ_rate & timecost
df_rna = data_rna1()
df_rna[df_rna['auditor_tag']=='Ewally']=df_rna[df_rna['auditor_tag']=='Ewally'].replace(to_replace=1,value=2)
df_rna

### rna succ rete

In [ ]:
df_rna_succ_week=df_rna[df_rna['type']=='week'].pivot_table(index=['period','rna_level','rna_type'],
                                                            values=['total','suc','pending'],
                                                            aggfunc=np.sum,
                                                            fill_value=0)
df_rna_succ_week['succ_rate']=df_rna_succ_week['suc']/(df_rna_succ_week['total']-df_rna_succ_week['pending'])
df_rna_succ_week.drop(columns=['suc','pending','total'],inplace=True)
df_rna_succ_week=df_rna_succ_week.xs('succ_rate',axis=1).unstack(0).fillna(0).sort_index(axis=1,ascending=False)
df_rna_succ_week

In [ ]:
df_rna_succ_month=df_rna[df_rna['type']=='month'].pivot_table(index=['period','rna_level','rna_type'],
                                                            values=['total','suc','pending'],
                                                            aggfunc=np.sum,
                                                            fill_value=0)
df_rna_succ_month['succ_rate']=df_rna_succ_month['suc']/(df_rna_succ_month['total']-df_rna_succ_month['pending'])
df_rna_succ_month.drop(columns=['suc','pending','total'],inplace=True)
df_rna_succ_month=df_rna_succ_month.xs('succ_rate',axis=1).unstack(0).fillna('-').sort_index(axis=1,ascending=False)
df_rna_succ_month

### rna overview

In [ ]:
#compute the succ_rate/closed_rate/unclosed_cnt of Audit-ISV distinguished by rna_level
df_rna_month=df_rna[(df_rna['type']=='month') & (df_rna['rna_type']=='INDIVIDUAL')].pivot_table(index=['rna_level','auditor_tag','period'],aggfunc=np.sum,fill_value=0)
#     sort=True,'rna_level',
# .sort_index(axis=1,ascending=False).xs('sum',axis=1)
df_rna_month['calc_case_cnt']=df_rna_month['total']-df_rna_month['pending']
# df_rna_month['succ_rate']=df_rna_month.apply(lambda x: '%.2f%%' % ((x['suc']*100)/x['calc_case_cnt']),axis=1)
df_rna_month['succ_rate']=df_rna_month['suc']/df_rna_month['calc_case_cnt']
# df_rna_month['succ_rate_ofuser']=df_rna_month.apply(lambda x: '%.2f%%' % ((x['suc_cnt']*100)/x['user']),axis=1)
df_rna_month['succ_rate_ofuser']=df_rna_month['suc_cnt']/df_rna_month['user']
df_rna_month['avg_case_peruser']=round(df_rna_month['total']/df_rna_month['user'],2)
df_rna_month['avg_deal_min']=round(df_rna_month['total_deal_min']/(df_rna_month['suc']+df_rna_month['fail']),2)
df_rna_month.drop(axis=1,columns=['fail','fail_cnt','suc', 'suc_cnt','total','total_deal_min'],inplace=True)
df_rna_month=df_rna_month.sort_index(axis=1,ascending=True)
df_rna_month

In [ ]:
df_rna_week=df_rna[(df_rna['type']=='week') & (df_rna['rna_type']=='INDIVIDUAL')].pivot_table(index=['rna_level','auditor_tag','period'],aggfunc=np.sum,fill_value=0)
#     sort=True,'rna_level',
# .sort_index(axis=1,ascending=False).xs('sum',axis=1)
df_rna_week['calc_case_cnt']=df_rna_week['total']-df_rna_week['pending']
# df_rna_week['succ_rate']=df_rna_week.apply(lambda x: '%.2f%%' % ((x['suc']*100)/x['calc_case_cnt']),axis=1)
df_rna_week['succ_rate']=df_rna_week['suc']/df_rna_week['calc_case_cnt']
# df_rna_week['succ_rate_ofuser']=df_rna_week.apply(lambda x: '%.2f%%' % ((x['suc_cnt']*100)/x['user']),axis=1)
df_rna_week['succ_rate_ofuser']=df_rna_week['suc_cnt']/df_rna_week['user']
df_rna_week['avg_case_peruser']=round(df_rna_week['total']/df_rna_week['user'],2)
df_rna_week['avg_deal_min']=round(df_rna_week['total_deal_min']/(df_rna_week['suc']+df_rna_week['fail']),2)
df_rna_week=df_rna_week.drop(axis=1,columns=['fail','fail_cnt','suc', 'suc_cnt','total','total_deal_min'])
df_rna_week=df_rna_week.sort_index(axis=1,ascending=True)
df_rna_week

## DEPOSIT & WITHDRAW

In [ ]:
@db_util.sql_to_df('ofp_mysql')
def data_deposit1():
    """succ_rate in first depo & withd"""
    sql = """    
    select
        'week' as type
        , date_format(subdate(date(created_at),if(date_format(date(created_at),'%%w')=0,7,date_format(date(created_at),'%%w'))-1),'%%Y-%%m-%%d') as period
        , record_type
        , balance_code
        , if(rnk=1,'first','not_first') as "rnk"
        -- , bank_name
        , bank_code
        , count(DISTINCT id) as "case"
        , count(DISTINCT if(record_status='SUCCESS',id,null)) as "succ"
        , count(DISTINCT if(record_status='FAIL',id,null)) as "fail"
        , count(DISTINCT if(record_status in ('NEDD_ADUIT','PAYED'),id,null)) as "pending"
        , sum(if(record_status in ('SUCCESS','FAIL'),TIMESTAMPDIFF(hour,created_at,finished_at),0)) as deal_hour
        , max(amount) max_amt
        , sum(amount) as total_amt
        , max(if(record_status='SUCCESS',amount,null)) succ_max_amt
        , sum(if(record_status='SUCCESS',amount,0)) as succ_amt
        , count(DISTINCT if(record_status='SUCCESS',cid,null)) as "succ_user"
     from
    (
        select a.*,row_number() over (partition by cid,record_type order by created_at) as rnk
        from
        (
            select DISTINCT m1.id,m1.cid,
            if(m1.finished_at<m1.created_at,m1.finished_at,m1.created_at) created_at,
            m1.finished_at,m1.record_type,m1.record_status,-- m1.bank_name,
            concat(
            case
              when m1.record_note like '%%ewally%%' then 'ewally'
              when m1.record_note in ('transferbalance-out','transferbalance-in') then 'internal_transfer'
              when m1.record_type='DEPOSIT' then m1.bank_code
              when m1.record_type='WITHDRAW' then ifnull(m1.pre_id,'manual')
            end,
            case when bank_id is null then '' else '_PLURAL' end ) bank_code
            ,m1.balance_code,m1.amount
            from retail.ofp_legal_record m1
            join retail.ofp_customer_ext m2 on m1.cid=m2.cid
            where m2.source != 'TEST' and m1.record_note != 'OTC' and m1.record_type in ('DEPOSIT','WITHDRAW')
            and m1.balance_code='CNY'
        )a order by 2,5,3
    )m1
    where created_at>=subdate(curdate(),if(date_format(curdate(),'%%w')=0,7,date_format(curdate(),'%%w'))-1) - interval '49' day and created_at<current_date
    group by 1,2,3,4,5,6
    
    union all
    
    select 
        'month' as type
        , date_format(created_at,'%%Y-%%m') as period
        , record_type
        , balance_code
        , if(rnk=1,'first','not_first') as "rnk"
        -- , bank_name
        , bank_code
        , count(DISTINCT id) as "case"
        , count(DISTINCT if(record_status='SUCCESS',id,null)) as "succ"
        , count(DISTINCT if(record_status='FAIL',id,null)) as "fail"
        , count(DISTINCT if(record_status in ('NEDD_ADUIT','PAYED'),id,null)) as "pending"
        , sum(if(record_status in ('SUCCESS','FAIL'),TIMESTAMPDIFF(hour,created_at,finished_at),0)) as deal_hour
        , max(amount) max_amt
        , sum(amount) as total_amt
        , max(if(record_status='SUCCESS',amount,null)) succ_max_amt
        , sum(if(record_status='SUCCESS',amount,0)) as succ_amt
        , count(DISTINCT if(record_status='SUCCESS',cid,null)) as "succ_user"
    from
    (
        select a.*,row_number() over (partition by cid,record_type order by created_at) as rnk
        from 
        (
            select DISTINCT m1.id,m1.cid,
            if(m1.finished_at<m1.created_at,m1.finished_at,m1.created_at) created_at,
            m1.finished_at,m1.record_type,m1.record_status,-- m1.bank_name,
            concat(
            case 
              when m1.record_note like '%%ewally%%' then 'ewally' 
              when m1.record_note in ('transferbalance-out','transferbalance-in') then 'internal_transfer' 
              when m1.record_type='DEPOSIT' then m1.bank_code 
              when m1.record_type='WITHDRAW' then ifnull(m1.pre_id,'manual')
            end,
            case when bank_id is null then '' else '_PLURAL' end ) bank_code
            ,m1.balance_code,m1.amount
            from retail.ofp_legal_record m1 
            join retail.ofp_customer_ext m2 on m1.cid=m2.cid
            where m2.source != 'TEST' and m1.record_note != 'OTC' and m1.record_type in ('DEPOSIT','WITHDRAW') 
            and m1.balance_code='CNY'
        )a order by 2,5,3
    )m1 
    where created_at>=date_format(curdate(),'%%Y-%%m-01') - interval '6' month 
    and created_at<date_format(curdate(),'%%Y-%%m-01')
    group by 1,2,3,4,5,6
    order by 1,2 desc,3
    """
    return sql

df_deposit = data_deposit1()
# df_deposit

In [ ]:
df_deposit_week=df_deposit[(df_deposit['type']=='week') & (df_deposit['bank_code'] != 'manual')].pivot_table(
    index=['record_type','bank_code','period'],
    values=['case','pending','deal_hour','succ','fail','total_amt','succ_amt','succ_user'],
    aggfunc=np.sum,
    margins=True,
    margins_name='TOTAL'
)
df_deposit_week['calc_case']=df_deposit_week['case']-df_deposit_week['pending']
df_deposit_week['calc_succ_rate']=df_deposit_week['succ']/df_deposit_week['calc_case']
# df_deposit_week.apply(lambda x: '%.2f%%' % ((x['succ']*100)/x['calc_case']),axis=1)
df_deposit_week['avg_amt_succ_case']=round(df_deposit_week['succ_amt']/df_deposit_week['succ'],2)
df_deposit_week['avg_deal_hour']=df_deposit_week['deal_hour']/(df_deposit_week['succ']+df_deposit_week['fail'])
df_deposit_week['amt_succ_rate']=df_deposit_week.apply(lambda x: '%.2f%%' % ((x['succ_amt']*100)/x['total_amt']),axis=1)
df_deposit_week['avg_amt_succ_user']=round(df_deposit_week['succ_amt']/df_deposit_week['succ_user'],2)
df_deposit_week['avg_case_succ_user']=round(df_deposit_week['calc_case']/df_deposit_week['succ_user'],2)
df_deposit_week['succ_amt']=round(df_deposit_week['succ_amt'],2)
df_deposit_week.drop(axis=1,columns=['deal_hour', 'fail','succ','total_amt','case'],inplace=True)
df_deposit_week

In [ ]:
df_deposit_month=df_deposit[(df_deposit['type']=='month') & ((df_deposit['bank_code']=='auto_PLURAL')|(df_deposit['bank_code']=='ewally')|(df_deposit['bank_code']=='manual_PLURAL'))].pivot_table(
    index=['record_type','bank_code','period'],
    values=['case','pending','deal_hour','succ','fail','total_amt','succ_amt','succ_user'],
    aggfunc=np.sum,
    margins=True,
    margins_name='TOTAL'
)
df_deposit_month['calc_case']=df_deposit_month['case']-df_deposit_month['pending']
df_deposit_month['calc_succ_rate']=df_deposit_month['succ']/df_deposit_month['calc_case']
# df_deposit_month.apply(lambda x: '%.2f%%' % ((x['succ']*100)/x['calc_case']),axis=1)
df_deposit_month['avg_amt_succ_case']=round(df_deposit_month['succ_amt']/df_deposit_month['succ'],2)
df_deposit_month['avg_deal_hour']=df_deposit_month['deal_hour']/(df_deposit_month['succ']+df_deposit_month['fail'])
df_deposit_month['amt_succ_rate']=df_deposit_month.apply(lambda x: '%.2f%%' % ((x['succ_amt']*100)/x['total_amt']),axis=1)
df_deposit_month['avg_amt_succ_user']=round(df_deposit_month['succ_amt']/df_deposit_month['succ_user'],2)
df_deposit_month['avg_case_succ_user']=round(df_deposit_month['calc_case']/df_deposit_month['succ_user'],2)
df_deposit_month['succ_amt']=round(df_deposit_month['succ_amt'],2)
df_deposit_month.drop(axis=1,columns=['deal_hour', 'fail','succ','total_amt','case'],inplace=True)
df_deposit_month

# file_loader = FileSystemLoader('/Users/PycharmProjects/common/templates/html')
#     jinja_env = Environment(loader=file_loader)
#     template = jinja_env.get_template("df_table_elem.html")
#     res = template.render(df=df_deposit_month, table_id="table1", cust_class="ui celled table")

In [ ]:
# try to color the df
df_deposit_month = df_deposit_month.sortlevel(0,axis=1)
df_deposit_month.style.bar(subset=df_deposit_month.loc[0:5,'succ_amt'],axis=0,color=['#d65f5f', '#5fba7d'],align='mid')
def show(df):
    return df.style.background_gradient(cmap='gray_r',axis=1)

def showlow(col):
    c = col == col.min()
    return ['background-color:red' if v else '' for v in c]
df_deposit_month.style.apply(showlow)
df_deposit_month.style.background_gradient(cmap='gray_r',axis=1,subset=pd.IndexSlice[:5,[''])
df_deposit_month.style.background_gradient()
df_deposit_month.iloc[:6,1]

## MONTHLY GOALS & PENDING

In [ ]:
@db_util.sql_to_df('ofp_mysql')
def data_p_and_l():
    """goal of each month"""
    sql = """    
    select distinct period,type,attr,value
    from retail.ba_goal_actual 
    where rna_area !='AMERICA' and period<=date_format(current_date,'%%Y-%%m-01')
    """
    return sql

result = data_p_and_l()
result = result.pivot_table(index = ['type','attr'],columns='period').stack(0).reset_index().fillna(0)
monthly_result = result.drop(['type','level_2'],axis=1)

# __CONVERT DFs TO HTML__
---
## READ DATA FROM DF

In [ ]:
from jinja2 import Template, FileSystemLoader, Environment
from sources.common.utils import cos_util
file_loader = FileSystemLoader('/Users/PycharmProjects/common/templates/html')  # change this to your absolute path
jinja_env = Environment(loader=file_loader)
template = jinja_env.get_template("df_table_elem.html")


today = str(dt.date.today())
# int((datetime(2015, 10, 30, 0, 0, 0) - datetime(1970, 1, 1, 8, 0, 0)).total_seconds())
this_week_end_date=str(dt.date.today()+relativedelta(weeks=-1,weekday=calendar.SUNDAY))
last_week_end_date=str(dt.date.today()+relativedelta(weeks=-2,weekday=calendar.SUNDAY))
three_weeks_ago=str(dt.date.today()+relativedelta(weeks=-3,weekday=calendar.MONDAY)) # 如果是周一改成-3
three_weeks_ago_end_date=str(dt.date.today()+relativedelta(weeks=-3,weekday=calendar.SUNDAY))
four_weeks_ago=str(dt.date.today()+relativedelta(weeks=-4,weekday=calendar.MONDAY)) # 如果是周一改成-4
four_weeks_ago_end_date=str(dt.date.today()+relativedelta(weeks=-4,weekday=calendar.SUNDAY))

grafana_timestamp_from = int((pd.to_datetime(dt.date.today())+relativedelta(weeks=-4,weekday=calendar.MONDAY)).timestamp() * 1000)
# ！！change to -4,+2419199 if it's Monday, +pd.Timedelta(days=-1)
grafana_timerange = "from=%d&to=%d" % (grafana_timestamp_from, grafana_timestamp_from + 86400 * 28 *1000 - 1)
# grafana_timerange = 'from=1632700800000&to=1635119999000'

In [ ]:
# three_weeks_ago=str(dt.date.today()+relativedelta(weeks=-3,weekday=calendar.MONDAY))
# three_weeks_ago_end_date=str(dt.date.today()+relativedelta(weeks=-3,weekday=calendar.SUNDAY))
# four_weeks_ago=str(dt.date.today()+relativedelta(weeks=-4,weekday=calendar.MONDAY))
# four_weeks_ago_end_date=str(dt.date.today()+relativedelta(weeks=-4,weekday=calendar.SUNDAY))
# print(three_weeks_ago,three_weeks_ago_end_date,four_weeks_ago,four_weeks_ago_end_date)

## BEAUTIFY

In [ ]:
# highligt when mouse hovering
from IPython.display import HTML
def hover(hover_color="#ffffcc"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])
hover_style = [hover(),
        dict(selector="th", props=[("font-size", "100%"),("text-align", "center")]),
        dict(selector="caption", props=[("caption-side", "bottom")])]
# html = (df.style.set_table_styles(hover_style).set_caption("Hover to highlight."))
# html


In [ ]:
# df_transaction_week.pd.IndexSlice[:,2:]

In [ ]:
# list(df_deposit_week.columns[1:])

In [ ]:
# list(df_channel_week.index)

In [ ]:
def poorneg(val):
    color='green' if val >0 else 'red'
    return 'color:{}'.format(color)

In [ ]:
# set all the df into the right place
# .background_gradient(cmap=sns.light_palette("green", as_cmap=True),subset=df_deposit_week.columns[2])
# .highlight_min().highlight_max()
# .hide_index()
# .background_gradient()
# df_first_trade_week
# pd.IndexSlice[0:5,['succ_amt']]
# .iloc[0:6,2]
# formatter={'calc_succ_rate':"{:.0f}",'amt_succ_rate':"{:.0f}"},
table_attributes='class="ui celled structured table very compact" border="1"'
table_attributes_1='class="ui celled structured collapsing table very compact" border="1"'

res1 = df_transaction_week.style.format(na_rep="-",precision=2,thousands=',',decimal='.') \
.set_table_styles(hover_style) \
.to_html(table_attributes=table_attributes)
res2 = df_transaction_month.style.format(na_rep="-",precision=2,thousands=',',decimal='.') \
.set_table_styles(hover_style) \
.to_html(table_attributes=table_attributes)
# .background_gradient(cmap=sns.<span class="mark">diverging_palette</span>(150, 10, s=80, l=55,as_cmap=True),subset=pd.IndexSlice[(slice[1:]),2:]) \

res3 = df_balance_week.style.format(na_rep="-",precision=2,thousands=',',decimal='.') \
.set_table_styles(hover_style).to_html(table_attributes= table_attributes)
res4 = df_balance_month.style.format(na_rep="-",precision=2,thousands=',',decimal='.') \
.set_table_styles(hover_style).to_html(table_attributes= table_attributes)
# ,subset=pd.IndexSlice[:,1:]；.applymap(poorneg) \
res5 = df_conv_detail_ov_week.style.format(na_rep="-",precision=2,thousands=',',decimal='.').set_table_styles(hover_style).to_html(table_attributes= table_attributes)
res6 = df_conv_detail_ov_month.style.format(na_rep="-",precision=2,thousands=',',decimal='.').set_table_styles(hover_style).to_html(table_attributes= table_attributes)

res7 = df_order_otc_week.style.format(na_rep="-",precision=2,thousands=',',decimal='.').set_table_styles(hover_style).to_html(table_attributes= table_attributes)
res8 = df_order_otc_month.style.format(na_rep="-",precision=2,thousands=',',decimal='.').set_table_styles(hover_style).to_html(table_attributes= table_attributes)

res9 = df_trans_type_week.style.format(na_rep="-",precision=2,thousands=',',decimal='.').set_table_styles(hover_style).to_html(table_attributes= table_attributes)
res10 = df_trans_type_month.style.format(na_rep="-",precision=2,thousands=',',decimal='.').set_table_styles(hover_style).to_html(table_attributes= table_attributes)

res11 = df_symbol_week_top.style.format(na_rep="-",precision=2,thousands=',',decimal='.').set_table_styles(hover_style).to_html(table_attributes= table_attributes_1)
res12 = df_symbol_month_top.style.format(na_rep="-",precision=2,thousands=',',decimal='.').set_table_styles(hover_style).to_html(table_attributes= table_attributes_1)

res13 = df_rna_week.style \
.background_gradient(cmap=sns.diverging_palette(10,150,s=70, l=60,as_cmap=True),subset=pd.IndexSlice[(1,'IDWall',slice(None)), ['succ_rate','succ_rate_ofuser','user']]) \
.background_gradient(cmap=sns.diverging_palette(10,150,s=70, l=60,as_cmap=True),subset=pd.IndexSlice[(2,'Ewally',slice(None)), ['succ_rate','succ_rate_ofuser','user']]) \
.format({'succ_rate_ofuser':"{:.2%}",'succ_rate':"{:.2%}"}) \
.format(na_rep="-",precision=2,thousands=',',decimal='.') \
.set_table_styles(hover_style) \
.to_html(table_attributes= table_attributes_1)
res14 = df_rna_month.style \
.background_gradient(cmap=sns.diverging_palette(10,150,s=70, l=60,as_cmap=True),subset=pd.IndexSlice[(1,'IDWall',slice(None)), ['succ_rate','succ_rate_ofuser','user']]) \
.background_gradient(cmap=sns.diverging_palette(10,150,s=70, l=60,as_cmap=True),subset=pd.IndexSlice[(2,'Ewally',slice(None)), ['succ_rate','succ_rate_ofuser','user']]) \
.format({'succ_rate_ofuser':"{:.2%}",'succ_rate':"{:.2%}"}) \
.format(na_rep="-",precision=2,thousands=',',decimal='.') \
.set_table_styles(hover_style) \
.to_html(table_attributes= table_attributes_1)


res15 = df_deposit_week.style \
.background_gradient(cmap=sns.diverging_palette(10,150,s=70, l=60,as_cmap=True),subset=pd.IndexSlice[('DEPOSIT','auto_PLURAL',slice(None)), ['succ_amt',"calc_case",'avg_amt_succ_case','avg_amt_succ_user','avg_case_succ_user']]) \
.background_gradient(cmap=sns.diverging_palette(10,150,s=70, l=60,as_cmap=True),subset=pd.IndexSlice[(['WITHDRAW'],'auto_PLURAL',slice(None)), ['succ_amt',"calc_case",'avg_amt_succ_case','avg_amt_succ_user','avg_case_succ_user','calc_succ_rate']]) \
.format({'calc_succ_rate':"{:.2%}"}).format(na_rep="-",precision=2,thousands=',',decimal='.') \
.set_table_styles(hover_style) \
.to_html(table_attributes= table_attributes)
res16 = df_deposit_month.style \
.background_gradient(cmap=sns.diverging_palette(10,150,s=70, l=60,as_cmap=True),subset=pd.IndexSlice[('DEPOSIT','auto_PLURAL',slice(None)), ['succ_amt',"calc_case",'avg_amt_succ_case','avg_amt_succ_user','avg_case_succ_user']]) \
.background_gradient(cmap=sns.diverging_palette(10,150,s=70, l=60,as_cmap=True),subset=pd.IndexSlice[('WITHDRAW','auto_PLURAL',slice(None)), ['succ_amt',"calc_case",'avg_amt_succ_case','avg_amt_succ_user','avg_case_succ_user','calc_succ_rate']]) \
.format({'calc_succ_rate':"{:.2%}"}).format(na_rep="-",precision=2,thousands=',',decimal='.') \
.set_table_styles(hover_style) \
.to_html(table_attributes= table_attributes)

res17 = df_channel_week.style.format(na_rep="-",thousands=',',decimal='.').format({'a.reg2rna1_succ':"{:.2%}",
                                      'b.reg2depo_succ':"{:.2%}",
                                      'c.reg2trade_succ':"{:.2%}",
                                      'd.reg2try_rna1(try/reg)':"{:.2%}",
                                      'e.rna_succ(succ/try)':"{:.2%}",
                                      'f.reg2try_depo(try/reg)':"{:.2%}",
                                      'g.depo_succ(succ/try)':"{:.2%}"
                                     }).set_table_styles(hover_style).to_html(table_attributes= table_attributes)
res18 = df_channel_month.style.format(na_rep="-",thousands=',',decimal='.').format({'a.reg2rna1_succ':"{:.2%}",
                                      'b.reg2depo_succ':"{:.2%}",
                                      'c.reg2trade_succ':"{:.2%}",
                                      'd.reg2try_rna1(try/reg)':"{:.2%}",
                                      'e.rna_succ(succ/try)':"{:.2%}",
                                      'f.reg2try_depo(try/reg)':"{:.2%}",
                                      'g.depo_succ(succ/try)':"{:.2%}"
                                     }).set_table_styles(hover_style).to_html(table_attributes=table_attributes)

res19 = df_rna_succ_week.style.format(na_rep="-",thousands=',',decimal='.').format("{:.2%}").set_table_styles(hover_style).to_html(table_attributes= table_attributes_1)
res20 = df_rna_succ_month.style.format(na_rep="-",thousands=',',decimal='.').format("{:.2%}").set_table_styles(hover_style).to_html(table_attributes= table_attributes_1)
# ,"avg_amt_succ_case" manual_PLURAL pd.IndexSlice[pd.IndexSlice[:,2],:]
# use slicing of this rule to part 
# df_deposit_week.style.background_gradient(cmap=sns.light_palette("green", as_cmap=True),\
#                      subset=pd.IndexSlice[('DEPOSIT','ewally',slice(None)), ['succ_amt',"avg_amt_succ_case"]])
# ['a.reg2rna1_succ','b.reg2depo_succ','c.reg2trade_succ','d.reg2try_rna1(try/reg)','e.rna_succ(succ/try)','f.reg2try_depo(try/reg)','g.depo_succ(succ/try)']

# end with </div><i class="fa fa-lightbulb-o "></i>

## EMBED TEMPLATE
_pass parameters_

In [ ]:
template = jinja_env.get_template("weekly_report.html")
res = template.render(
    table_html1=res1, 
    table_html2=res2,
    table_html3=res3,
    table_html4=res4,
    table_html5=res5,
    table_html6=res6,
    table_html7=res7,
    table_html8=res8,
    table_html9=res9,
    table_html10=res10,
    table_html11=res11,
    table_html12=res12,
    table_html13=res13,
    table_html14=res14,
    table_html15=res15,
    table_html16=res16,
    table_html17=res17,
    table_html18=res18,
    table_html19=res19,
    table_html20=res20,
    
    today=today,
    this_week=this_week,
    last_week=last_week,
    this_month=this_month,
    last_month=last_month,
    last_day=last_day,
    first_day=first_day,
    this_week_end_date=this_week_end_date,
    last_week_end_date=last_week_end_date,
    grafana_timerange=grafana_timerange,
    three_weeks_ago=three_weeks_ago,
    three_weeks_ago_end_date=three_weeks_ago_end_date,
    four_weeks_ago=four_weeks_ago,
    four_weeks_ago_end_date=four_weeks_ago_end_date
)

# __UPLOAD TO SERVER__
---
_start from TencentCloud personal server and then move to company's AWS server_

In [ ]:
# relative date = last week date = this week variable in this script
# url, cos_resp = cos_util.put_object("/prod/weekly_report/%s.html" % this_week, res)
url = report_util.push("weekly_report/%s.html" % this_week, res)
print(url)
from sources.common.utils import notify_util
notify_util.send("personal-dingbangchu", "请阅上周周报。", url=url)
# notify_util.send("bi-report", "请阅上周周报。", url=url)

# __AUTO SCREENSHOT__
---
- _for static png/pdf_
- _for international collegues with unstable network or temporary authority issues_
- _**unrelease function: in test stage**_


In [ ]:
import asyncio
from pyppeteer import launch

async def screenshot():
    browser = await launch({'headless': False})
    page = await browser.newPage()
    await page.setViewport({"width": 1920, "height": 1080, 'deviceScaleFactor': 2})

    await page.goto(url='https://bi.cos.ap-beijing.myqcloud.com/prod/weekly_report/2023-06-01.html', options={"timeout": 90000})

    await page.evaluate("""{window.scrollBy(0, document.body.scrollHeight);}""")
    await page.waitFor(10000)
    await page.screenshot({'path': 'example.jpg', 'type': 'jpeg', 'quality': 100, 'fullPage': True})
    await browser.close()

if __name__ == '__main__':
    asyncio.get_event_loop().run_until_complete(screenshot())

In [ ]:
url, cos_resp = cos_util.put_object("/prod/weekly_report/2023-06-01.html", res)
print(url)

# __SAVE TO A EXCEL ATTACHMENT__
---
- _for archive in kownlegde repository_
- _for reproduce_
- _for more friendly format fot marketing/non-tech department_
- _unrelease function: in test stage_

In [ ]:
writer = pd.ExcelWriter('./ofp_weekly_report_{date}.xlsx'.format(date=today.strftime('%Y%m%d')), engine='openpyxl')

def output_exl(dfs,sheet_name,k):
    i = 0
    for df in dfs:
        df.to_excel(excel_writer=writer, sheet_name=sheet_name,startrow=i,index=False,freeze_panes=(0,1))
        row = len(df.index)
        i += row + 4 
    ws = writer.sheets[sheet_name]
    for j in range(len(dfs[k].columns)):
        if j == 0:
            ws.column_dimensions[ascii_uppercase[j]].width = 30
        else:
            ws.column_dimensions[ascii_uppercase[j]].width = 20

output_exl(dfs = [df_transaction_week,df_balance_week,df_first_trade_week,df_order_otc_week],sheet_name = '周概览',k=1)
output_exl(dfs = [df_transaction_month,df_balance_month,df_first_trade_month,df_order_otc_month],sheet_name = '月概览',k=1)

output_exl(dfs = [df_trans_type],sheet_name = '交易类型',k=0)
output_exl(dfs = [df_symbol],sheet_name = '交易币对',k=0)

# output_exl(dfs = [df_order_device],sheet_name = '下单时终端',k=0)
output_exl(dfs = [df_deposit],sheet_name = '存款',k=0)
output_exl(dfs = [df_conv_detail],sheet_name = '获客转化漏斗',k=0)
output_exl(dfs = [df_rna],sheet_name = 'rna',k=0)
output_exl(dfs = [monthly_result],sheet_name = '各月月度盈亏目标',k=0)

writer.save()
print('check file: ./ofp_weekly_report_{date}.xlsx'.format(date=today.strftime('%Y%m%d')))

In [ ]:
#UTC time：2020-04-24 07:18:07.677000+00:00
#output：20200424151807
n_time = datetime.datetime.strptime('2023-10-27 15:30:00.000000+00:00','%Y-%m-%d %H:%M:%S.%f+00:00')  
utctime = datetime.datetime(n_time.year,n_time.month,n_time.day,n_time.hour,n_time.minute,n_time.second,tzinfo=timezone('UTC'))
lt = utctime.astimezone(timezone('Asia/Shanghai'))
llt = lt.strftime("%Y%m%d%H%M%S")
print(llt)